In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [4]:
pip install sdv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 3.9 MB/s 
     |████████████████████████████████| 61 kB 363 kB/s 
     |████████████████████████████████| 47 kB 4.7 MB/s 
     |████████████████████████████████| 139 kB 33.4 MB/s 
     |████████████████████████████████| 1.6 MB 50.6 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 9.4 MB 50.3 MB/s 
     |████████████████████████████████| 295 kB 75.0 MB/s 
     |████████████████████████████████| 965 kB 56.9 MB/s 
     |████████████████████████████████| 662 kB 57.1 MB/s 
     |████████████████████████████████| 280 kB 62.5 MB/s 
     |████████████████████████████████| 15.3 MB 48.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Attempting uninstall: psutil
    Found existing installation: p

In [5]:
import glob
def preprocessing(input_path):
    all_input_list = sorted(glob.glob(input_path))
    train = pd.DataFrame()
    for datapath in all_input_list:
        data = pd.read_csv(datapath) 
        data['obs_time'] = data.index % 24 
        df = abs(data)
        df['시간당총광량'] = df['시간당청색광량']+df['시간당백색광량']+df['시간당적색광량']
        col_list = df.columns
        for i in range(0,len(col_list)):
            col = col_list[i]    
            if '누적' in col : 
                df[col] = df.groupby((df.obs_time == 0).cumsum()).agg(col_list[i-1]).cumsum()   
            train = pd.concat([train,df])
    return train

In [6]:
trainx = preprocessing('drive/MyDrive/dacon/gan/상추/train_input/*.csv')

In [7]:
day0 = trainx[trainx['DAT']==0].reset_index(drop=True)
day0 = day0[['obs_time','내부온도관측치','내부습도관측치','co2관측치','ec관측치','시간당분무량','시간당백색광량','시간당적색광량','시간당청색광량']]
day0.describe()

,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,시간당백색광량,시간당적색광량,시간당청색광량
count,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000
mean,11.500000,25.781004,54.912360,533.833783,1.273970,430.609479,6765.408458,1309.564887,856.852189
std,6.922508,4.361793,12.257845,144.127605,0.932456,491.308581,9450.283141,2653.722924,1938.172567
min,0.000000,0.000000,0.000000,60.400000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.750000,23.595463,49.183641,446.345833,0.401136,0.000000,0.000000,0.000000,0.000000
50%,11.500000,26.280460,56.510000,505.883333,1.212455,242.355000,0.000000,0.000000,0.000000
75%,17.250000,28.717917,61.310767,578.400000,2.011237,769.000000,18255.190000,976.332000,135.110625
max,23.000000,39.158823,81.900001,1200.000000,3.034100,2735.210000,120000.000000,9928.800000,18570.857500


In [9]:
# epochs 비교
model1 = CTGAN(epochs = 1000)
model2 = CTGAN(epochs = 500)
model3 = CTGAN(epochs = 100)
model1.fit(day0)
model2.fit(day0)
model3.fit(day0)
ctgan1 = model1.sample(100,randomize_samples=0)
ctgan2 = model2.sample(100,randomize_samples=0)
ctgan3 = model3.sample(100,randomize_samples=0)

In [10]:
ctgan1.describe()

,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,시간당백색광량,시간당적색광량,시간당청색광량
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000
mean,12.300000,25.082651,54.324952,527.600000,1.380000,404.740000,7321.020000,1588.44000,1220.180000
std,6.989892,3.898884,13.346068,118.885651,0.951129,477.871578,8516.845308,2677.25641,2003.684436
min,0.000000,15.787870,15.379849,245.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,7.000000,22.072181,46.748015,449.000000,1.000000,4.000000,0.000000,0.00000,0.000000
50%,13.500000,25.249202,57.494733,514.000000,1.000000,122.500000,101.500000,13.00000,8.000000
75%,18.000000,27.981386,60.085151,580.000000,2.000000,767.000000,18099.250000,1373.50000,2614.750000
max,23.000000,32.096905,81.454696,1098.000000,3.000000,1953.000000,18415.000000,9220.00000,8769.000000


In [11]:
ctgan2.describe()

,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,시간당백색광량,시간당적색광량,시간당청색광량
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000
mean,12.070000,26.254630,54.301366,518.000000,1.260000,456.260000,7237.610000,1309.940000,995.54000
std,6.488926,4.292309,13.740221,139.499683,1.031034,484.185093,8489.349473,2590.623426,2039.97269
min,0.000000,14.411237,14.188004,91.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,6.000000,24.329396,47.206449,448.000000,0.000000,3.750000,0.000000,0.000000,0.00000
50%,12.500000,27.076480,56.321469,507.500000,1.000000,243.500000,40.000000,7.000000,5.00000
75%,18.000000,29.367164,61.856615,543.000000,2.000000,770.250000,18140.000000,1014.250000,737.50000
max,22.000000,33.710033,78.526540,1183.000000,3.000000,1461.000000,18451.000000,9680.000000,8813.00000


In [12]:
ctgan3.describe()

,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,시간당백색광량,시간당적색광량,시간당청색광량
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,12.260000,26.374072,56.463580,562.840000,0.970000,428.090000,6643.810000,1049.330000,702.290000
std,6.650921,4.215657,12.378052,163.730977,1.019556,463.117545,8561.607321,2295.582926,1850.399702
min,0.000000,14.976791,16.180806,329.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,25.097141,52.054349,464.750000,0.000000,0.000000,19.750000,0.000000,0.000000
50%,12.500000,26.589354,56.793644,524.000000,1.000000,247.500000,112.000000,6.000000,4.500000
75%,19.000000,29.238893,62.721788,576.250000,2.000000,768.250000,18061.000000,952.750000,21.500000
max,23.000000,34.296629,81.900001,1200.000000,3.000000,1901.000000,18615.000000,9630.000000,9593.000000


In [14]:
day0[day0['obs_time']==12].describe()

,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,시간당백색광량,시간당적색광량,시간당청색광량
count,448.0,448.000000,448.000000,448.000000,448.000000,448.00000,448.000000,448.000000,448.000000
mean,12.0,28.168015,52.396259,550.849519,1.351636,570.62750,14332.313214,1575.960600,1202.624429
std,0.0,2.726763,11.346562,165.354290,0.955384,473.19584,7493.392317,2580.119913,1639.936623
min,12.0,21.115000,17.696296,391.118644,0.000000,0.00000,0.000000,0.000000,0.000000
25%,12.0,26.627177,46.120834,440.016667,0.548308,125.35000,18177.837500,0.000000,0.000000
50%,12.0,27.740000,53.598113,518.416667,1.251020,762.57000,18255.190000,0.000000,0.000000
75%,12.0,29.856666,57.526250,583.387500,2.083267,769.00000,18255.190000,1327.149600,3101.670000
max,12.0,33.500000,75.691667,1200.000000,2.921124,1473.70000,18255.190000,8770.440000,4621.175000


In [15]:
ctgan1[ctgan1['obs_time']==12].describe()

,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,시간당백색광량,시간당적색광량,시간당청색광량
count,2.0,2.000000,2.000000,2.000000,2.0,2.000000,2.000000,2.000000,2.000000
mean,12.0,27.078931,47.953277,426.500000,1.0,383.500000,14.000000,7335.500000,1867.000000
std,0.0,3.068109,13.164235,13.435029,0.0,542.350901,5.656854,1863.226368,2619.123518
min,12.0,24.909450,38.644758,417.000000,1.0,0.000000,10.000000,6018.000000,15.000000
25%,12.0,25.994190,43.299017,421.750000,1.0,191.750000,12.000000,6676.750000,941.000000
50%,12.0,27.078931,47.953277,426.500000,1.0,383.500000,14.000000,7335.500000,1867.000000
75%,12.0,28.163671,52.607537,431.250000,1.0,575.250000,16.000000,7994.250000,2793.000000
max,12.0,29.248412,57.261797,436.000000,1.0,767.000000,18.000000,8653.000000,3719.000000


In [16]:
ctgan2[ctgan2['obs_time']==12].describe()

,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,시간당백색광량,시간당적색광량,시간당청색광량
count,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,12.0,27.286924,58.792519,509.555556,1.111111,834.000000,15253.000000,2670.888889,1429.555556
std,0.0,2.099767,12.400952,60.116784,0.781736,429.516007,6384.474469,3979.866877,2913.287538
min,12.0,23.619750,37.423941,403.000000,0.000000,220.000000,0.000000,0.000000,0.000000
25%,12.0,26.411846,53.414384,489.000000,1.000000,765.000000,18065.000000,0.000000,0.000000
50%,12.0,27.071175,55.013763,503.000000,1.000000,775.000000,18137.000000,1005.000000,6.000000
75%,12.0,28.039976,71.412981,534.000000,2.000000,1165.000000,18297.000000,2255.000000,715.000000
max,12.0,30.818654,76.834249,613.000000,2.000000,1430.000000,18430.000000,9680.000000,8533.000000


In [17]:
ctgan3[ctgan3['obs_time']==12].describe()

,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,시간당백색광량,시간당적색광량,시간당청색광량
count,2.0,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000,2.0
mean,12.0,28.120917,49.906305,449.000000,1.000000,392.500000,9235.000000,1.50000,0.0
std,0.0,4.623416,8.148935,15.556349,1.414214,543.765115,12948.539377,2.12132,0.0
min,12.0,24.851669,44.144137,438.000000,0.000000,8.000000,79.000000,0.00000,0.0
25%,12.0,26.486293,47.025221,443.500000,0.500000,200.250000,4657.000000,0.75000,0.0
50%,12.0,28.120917,49.906305,449.000000,1.000000,392.500000,9235.000000,1.50000,0.0
75%,12.0,29.755541,52.787388,454.500000,1.500000,584.750000,13813.000000,2.25000,0.0
max,12.0,31.390166,55.668472,460.000000,2.000000,777.000000,18391.000000,3.00000,0.0


In [8]:
## sdv ##
from sdv.tabular import GaussianCopula # Importing 'GaussianCoupla' Model
from sdv.tabular import CTGAN # Importing 'CTGAN' Model
from sdv.tabular import CopulaGAN # Importing 'CopulaGAN' Model
from sdv.evaluation import evaluate # Importing 'evaluate' Class for Synthetic Dataset Evaluation

## seed ##
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import random
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)
torch.cuda.manual_seed(1)
torch.cuda.manual_seed_all(1)

In [ ]:
# CTGAN
from ctgan import CTGANSynthesizer
import pandas as pd
import warnings
import joblib
import os
#from gan_feature.gan_make_dataset import make_data
#from gan_feature.gan_preprocessing import *
warnings.filterwarnings('ignore')

input_path = "./data/train_input/*.csv"
target_path = "./data/train_target/*.csv"
generated_path = "./generated_data/"


class CreateGE():
    def __init__(self):
        self.gan_model = CTGANSynthesizer()
        self.predict_model = joblib.load("base.pkl")
        self.raw_data = make_dataset(input_path, target_path)
        self.discrete_col = ['obs_time']
    
    @property        
    def generated_data(self):
        print("Start generated_data")
        for i in range(28):
            print(f"=======Start Day{i}===========")
            raw_data = self.raw_data[self.raw_data["DAT"] == i]
            self.gan_model.fit(raw_data, self.discrete_col, epochs=100)
            new_data = self.gan_model.sample(240)
            print(new_data.shape)
            new_data.to_csv(os.path.join(generated_path, f'generate_day{i}.csv'), index=False)
            print(f"========End Day{i}=========\n\n")
    
    def growth_env(self):
        y_list = []
        for i in range(28):
            data = pd.read_csv(f"./generated_data/generate_day{i}.csv") # 동적변경
            data = data.groupby(['obs_time']).mean().reset_index()
            preprocessed_data = make_data(data)
            break
            # y_prediction = self.predict_model.predict(preprocessed_data)
            # y_list.append(y_prediction)
            # max(y_list) # y_preddiction중 max값 반환해서 저장할 예정
        
        
        
if __name__ == '__main__':
    cls = CreateGE()
    # cls.generated_data
    cls.growth_env()
